In [100]:
import copy
import collections as col
import math
################################################ класс вершин ################################################
class Vertex:
    # вершина и смежные с ней вершины и их веса
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}

    def __del__(self):
        del self.id
        del self.connectedTo
    
    # добавление смежной вершины
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight
    
    # получение веса с данной вершиной
    def getWeight(self, nbr):
        return self.connectedTo[nbr]
    
    # удаление вершины
    def popVertex(self, key):
        self.connectedTo.pop(key)
     
    # возвращает вершину если она есть в смежных
    def getVertex(self,n):
        return n in self.connectedTo

    
################################################ класс граф ################################################
class Graph:
    def __init__(self):
        self.vertList = {}
        self.oriented = False

    def addVertex(self,key):
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex
            
    def __contains__(self,n):
        return n in self.vertList
    
    def addEdge(self,v,e,w=0,o=False):
        if self.oriented == True:
            if v not in self.vertList:
                nv = self.addVertex(v)
            if e not in self.vertList:
                nv = self.addVertex(e)
            self.vertList[v].addNeighbor(e, w)
        else:
            if v not in self.vertList:
                nv = self.addVertex(v)
            if e not in self.vertList:
                nv = self.addVertex(e)
            self.vertList[v].addNeighbor(e, w)
            self.vertList[e].addNeighbor(v, w)
        
    def delEdge(self,v,e,w=0,o=False):
        if self.oriented == True:
            if self.vertList[v].getVertex(e):
                self.vertList[v].popVertex(e)
                print(f"in {v} delete {e}")
            else:
                print(f"error in {v}")
        else: 
            if self.vertList[v].getVertex(e) and self.vertList[e].getVertex(v):
                self.vertList[v].popVertex(e)
                self.vertList[e].popVertex(v)
                print(f"in {v} delete {e}\nin {e} delete {v}")
            else:
                print(f"error in {v} or {e}")
    
    def delVertex(self, key):
        if self.__contains__(key):
            self.vertList.pop(key)
        print(f"Vertex: {key} is delete")
        for x in self.vertList:
            if self.vertList[x].getVertex(key):
                    self.vertList[x].popVertex(key)

    def getVertices(self):
        return self.vertList.keys()
    
    
    #@classmethod
    def read(self, filename):
        with open(filename) as text:
            n = int(text.read(2))
            ori = text.read(1)
            if ori == 1: self.oriented = True
            for i in range(n):
                self.addVertex(i)
            for line in text:
                lst = line.split()
                l = len(lst)
                lst = [int(item) for item in lst]
                if  l == 3:
                    self.addEdge(lst[0], lst[1], lst[2])
                elif l == 2:
                    self.addEdge(lst[0], lst[1])
    #
    def copyVert(self, new):
        for v in self.vertList:
            new[v] = copy.deepcopy(self.vertList[v])
        return new
    
    def write(self, filename):
        with open(filename, 'w') as text:
            n = text.write(f"{len(self.vertList)} ")
            ori = text.write(f'{self.oriented}\n')
            for v in self.vertList:
                for w in self.vertList[v].connectedTo:
                        text.write("%s %s %s" % (v, w, self.vertList[v].getWeight(w)) + '\n')
    
#     # в ширину
    def bfs (self, s):
        q = col.deque()
        q.append(str(s))
        visited = []
        p=[]
        for i in self.vertList:
            visited.append(i)
            p.append(i) 
            visited[i]=False
        visited[s]=True
        p[s] = -1
        
        while q:
            u = int(q.popleft())
            for to in self.vertList[u].connectedTo:
                if not visited[to]:
                    q.append(str(to))
                    visited[to]=True
                    p[to]=u 
                    
        for i in self.vertList:
            if i == s and not self.vertList[s].getVertex(s):
                visited[i] = False
            if not visited[i]:
                print([])
            else:
                path=[]
                v=i
                while v!=-1:
                    path.append(v)
                    v=p[v]
                print(path)
     
    # алгоритм Прима
    def prim (self):
        visited = []
        min_e = []
        best_e = []
        for i in self.vertList:
            visited.append(i) 
            min_e.append(i) 
            best_e.append(i) 
            visited[i] = False
            min_e[i] = math.inf
            
        min_e[0]=0
        
        for i in self.vertList:
            v = -1
            for u in self.vertList:
                if not visited[u] and (v == -1 or min_e[u] < min_e[v]):
                    v = u
            visited[v] = True
            if v != 0:
                print(f"{v} {best_e[v]}")
                
            for e in self.vertList[v].connectedTo:
                u=e
                w=self.vertList[v].connectedTo[e]
                if w<min_e[u]:
                    min_e[u]=w
                    best_e[u]=v
            
################################################ консоль ################################################
    # поиск в глубину 
    def dfs(self, u, v, visited, path):
        visited[u] = True
        path.append(u)
        if u == v:
            print(path)
        else:
            for i in self.vertList[u].connectedTo:
                if visited[i] == False:
                    self.dfs(i, v, visited, path)
        path.pop()
        visited[u] = False

    def vistAll(self, u,v):
        visited = []
        for i in self.vertList:
            visited.append(i) 
            visited[i]=False
        # массив для хранения путей
        path = []
        self.dfs(u, v, visited, path)
              
    def copyGraph(self, newGraph):
        for i in self.vertList:
            newGraph.addVertex(i)
        self.copyVert(newGraph.vertList)
        return newGraph
    
    # просмотр графа
    def lookAtGraph(self):
        for v in self.vertList:
            print("%s:" % v, end=' ')
            if len(self.vertList[v].connectedTo) == 0:
                    print("[]")
            else: 
                c=0
                for w in self.vertList[v].connectedTo:
                    c +=1
                    if  len(self.vertList[v].connectedTo)== c:
                        print("[%s:%s]" % (w, self.vertList[v].getWeight(w)))
                    else: 
                        print("[%s:%s]" % (w, self.vertList[v].getWeight(w)), end =', ') 


    def console(self):
        print("Welcome to Graph!\n\nChoose an action:\n 1. Add vertex\n 2. Delete vertex\n 3. Add edge\n 4. Delete edge\n 5. View\n 6. DFS\n 7. BFS\n")
        try:
            act = int(input('Enter action:'))
            if act == 1:
                v = int(input('Add vertex:'))
                self.addVertex(v)
                print("You added vertex!")
            elif act == 2:
                v = int(input('Delete vertex:'))
                self.delVertex(v)
                print("You delete vertex!")
            elif act == 3:
                v = int(input('Add vertex:'))
                e = int(input('Add adjacent vertex:'))
                w = int(input('Add weight or 0:'))
                self.addEdge(v,e,w)
                print(f"You added edge ({v},{e}) with weight {w}")
            elif act == 4:
                v = int(input('Delete vertex:'))
                e = int(input('Delete adjacent vertex:'))
                self.delEdge(v,e,w)
                print(f"You delete edge ({v},{e})")
            elif act == 5:
                self.lookAtGraph()
            elif act == 6:
                v = int(input('From vertex:'))
                e = int(input('To vertex:'))
                self.vistAll(v,e)
            elif act == 7:
                v = int(input('From vertex:'))
                self.vistAll(v)
            else:
                print("You entered the wrong action")
        except:
            print("You entered not a number")

##### Вывести полустепень исхода данной вершины орграфа.

##### Вывести все вершины графа, не смежные с данной.

##### Построить орграф, являющийся объединением двух заданных.

##### Вывести все пути из u в v.

##### Вывести кратчайшие пути из данной вершины в остальные

##### Алгоритм Прима